# 콘텐츠 마케팅팀을 대체하는 멀티 Agent
- Agent Monitoring & Observability 설정   
  https://docs.crewai.com/how-to/langfuse-observability#step-5-see-traces-in-langfuse

In [19]:
%pip install -qU python-dotenv beautifulsoup4 crewai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import getpass
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from crewai import Agent, Task, Crew
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

## LLM 선택

In [38]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

## 사용할 도구 정의
* Tavily를 사용한 온라인 검색
* BeautifulSoup를 사용한 HTML 구문 분석
* return_direct=False
    * 의미: 이 도구는 앱에 공개적으로 결과를 반환하지 않고 에이전트에 비공개로 결과를 반환합니다.

In [27]:
@tool("process_search_tool", return_direct=False)
def process_search_tool(url: str) -> str:
    """Used to process content found on the internet."""
    response = requests.get(url=url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup.get_text()

tools = [TavilySearchResults(max_results=1), process_search_tool]


* 참고 : 데모 및 비용 목적으로 Tavily를 사용한 온라인 검색의 max_results를 1로만 설정했습니다.   
        이렇게 하면 온라인 검색 에이전트가 여러 결과를 비교하는 대신 하나의 검색 결과만 연구하기 때문에 온라인 검색의 품질이 제한됩니다.

## online_researcher 에이전트 생성

In [28]:
online_researcher = Agent(
    role="온라인 리서처",
    goal="온라인에서 주제를 리서치",
    backstory="당신의 주요 역할은 정치, 기술, 건강, 문화, 글로벌 이벤트와 같은 "
              "다양한 분야에서 최신 및 가장 관련성 있는 "
              "트렌드 스토리를 인터넷에서 검색하는 데 능숙한 " 
              "지능형 온라인 리서치 어시스턴트 역할을 하는 것입니다. "
              "당신은 광범위한 온라인 뉴스 소스, 블로그, 소셜 미디어 플랫폼에 "
              "액세스하여 실시간 정보를 수집할 수 있는 역량을 보유하고 있습니다.",
    verbose=True,
    allow_delegation=True
)

## blog_manager 에이전트 생성

In [29]:
blog_manager = Agent(
    role="블로그 관리자",
    goal="블로그 기사 작성",
    backstory="당신은 블로그 관리자입니다. "
                "블로그 관리자의 역할에는 초기 초안을 청중을 참여시키고 "
                "성장시키는 세련되고 SEO 최적화된 블로그 기사로 변환하는 것을 " 
                "목표로 하는 몇 가지 중요한 책임이 포함됩니다. "
                "온라인 리서처가 제공한 초안을 시작으로 블로그 관리자는 "
                "블로그의 톤, 대상 청중 및 주제 목표와 일치하는지 확인하여 " 
                "콘텐츠를 철저히 이해해야 합니다. "
                "주요 책임은 다음과 같습니다.\n "
                "1. 콘텐츠 개선: 명확성, 흐름 및 참여를 개선하여 초안의 품질을 높입니다. "
                "여기에는 내러티브를 다듬고, 매력적인 헤더를 추가하고, "
                "기사가 독자 친화적이고 유익한지 확인하는 것이 포함됩니다.\n "
                "2. SEO 최적화: 검색 엔진 최적화를 위한 모범 사례 구현. "
                "여기에는 키워드 조사 및 통합, 메타 설명 최적화, "
                "URL 구조 및 헤딩 태그가 검색 엔진 결과에서 "
                "가시성을 향상하도록 보장하는 것이 포함됩니다.\n "
                "3. 규정 준수 및 모범 사례: 콘텐츠가 저작권법 및 광고의 진실을 포함한 "
                "법적 및 윤리적 기준을 준수하는지 확인합니다. "
                "블로그 관리자는 또한 콘텐츠 효과를 유지하고 향상시키기 위해 "
                "진화하는 SEO 전략과 블로깅 트렌드를 따라가야 합니다.\n "
                "4. 편집 감독: 모든 블로그 게시물에서 일관된 목소리와 품질을 유지하기 위해 "
                "작성자 및 기고자와 긴밀히 협력합니다. "
                "여기에는 콘텐츠 일정을 관리하고, 최적의 참여를 위해 게시물을 예약하고, "
                "마케팅 팀과 협력하여 홍보 활동을 지원하는 것도 포함될 수 있습니다.\n "
                "5. 분석 및 피드백 통합: 정기적으로 성과 지표를 검토하여 청중 참여와 선호도를 파악합니다. "
                "이 데이터를 사용하여 향후 콘텐츠를 개선하고 전반적인 블로그 전략을 최적화합니다.\n "
                "요약하자면, 블로그 관리자는 콘텐츠 품질을 향상하고, "
                "SEO 호환성을 보장하고, 블로그의 전략적 목표와 일치시켜 "
                "초기 연구와 최종 출판을 연결하는 데 중요한 역할을 합니다. "
                "이 직책에는 블로그의 온라인 존재감을 성공적으로 관리하고 성장시키기 위한 "
                "창의적, 기술적, 분석적 기술이 결합되어야 합니다.\n ",
    verbose=True,
    allow_delegation=True
)

## social_media_manager 에이전트 생성

In [30]:
social_media_manager = Agent(
    role="소셜 미디어 관리자",
    goal="트윗 쓰기",
    backstory="당신은 소셜 미디어 관리자입니다. "
                "특히, Twitter 콘텐츠를 관리하는 소셜 미디어 관리자의 역할에는 "
                "연구 초안을 청중에게 공감을 얻고 플랫폼 모범 사례를 준수하는 "
                "간결하고 매력적인 트윗으로 변환하는 것이 포함됩니다. "
                "온라인 연구자로부터 초안을 받으면 소셜 미디어 관리자는 몇 가지 중요한 기능을 담당합니다.\n "
                "1. 콘텐츠 요약: 초안의 핵심 메시지를 트윗으로 추출합니다. "
                "일반적으로 280자만 허용됩니다. "
                "이를 위해서는 메시지의 본질과 영향을 유지하면서 간결함에 집중해야 합니다.\n "
                "2. 참여 최적화: 참여를 극대화하는 트윗을 작성합니다. "
                "여기에는 설득력 있는 언어, 관련 해시태그, "
                "타겟 청중에게 공감을 얻는 시기적절한 주제를 "
                "전략적으로 사용하는 것이 포함됩니다.\n "
                "3. 규정 준수 및 모범 사례: 트윗이 언급, 해시태그 및 링크의 "
                "적절한 사용을 포함하여 Twitter의 지침과 모범 사례를 따르도록 합니다. " 
                "또한 윤리적 기준을 준수하고 잘못된 정보를 피하고 저작권 규범을 존중합니다.\n "
                "요약하자면, 소셜 미디어 관리자의 역할은 Twitter를 활용하여 "
                "정보를 효과적으로 전파하고, 팔로워와 소통하고, "
                "온라인에서 브랜드의 입지를 구축하는 데 중요합니다. " 
                "이 직책은 창의적인 커뮤니케이션 기술과 전략적 계획 및 "
                "분석을 결합하여 소셜 미디어 영향을 최적화합니다. ",
    verbose=True,
    allow_delegation=True
)

## content_marketing_manager 에이전트 생성

In [31]:
content_marketing_manager = Agent(
    role="콘텐츠 마케팅 관리자",
    goal="콘텐츠 마케팅 팀 관리",
    backstory="당신은 훌륭한 콘텐츠 마케팅 관리자입니다. "
                "당신의 주요 역할은 '블로그 관리자'의 각 게시물과 '소셜 미디어 관리자'가 "
                "작성한 트윗을 감독하고 게시를 승인하는 것입니다. "
                "작업을 검토하고 폭력적인 언어, 학대적인 콘텐츠 및 "
                "인종 차별적인 콘텐츠를 규제합니다.\n "
                "역량:\n "
                "편집 검토: 블로그 관리자와 소셜 미디어 관리자의 최종 초안을 분석하여 "
                "스타일 일관성, 주제적 정렬 및 전반적인 내러티브 흐름을 확인합니다.\n "
                "품질 보증: 뉴스 콘텐츠의 문법적 정확성, 사실적 정확성 및 저널리즘 표준 준수, " 
                "광고의 창의성 및 효과성에 대한 자세한 검사를 수행합니다.\n "
                "피드백 루프: 블로그 관리자와 소셜 미디어 관리자 모두에게 "
                "건설적인 피드백을 제공하여 콘텐츠 생성 및 프레젠테이션의 "
                "지속적인 개선을 위한 협업 환경을 촉진합니다. ",
    verbose=True,
    allow_delegation=True
)

## task 생성

In [32]:
task1 = Task(
    description="Agentic Behavior에 대한 보고서를 작성해 주세요. "
                "Agentic Behavior에 대한 조사가 끝나면  "
                "블로그 관리자에게 결과를 전달하여 최종 블로그 기사를 생성해 주세요. "
                "완료되면 소셜 미디어 관리자에게 전달하여 해당 주제에 대한 트윗을 작성해 주세요. ",
    expected_output="Agentic Behavior에 대한 보고",
    agent=online_researcher
)

In [33]:
task2 = Task(
    description="조사 결과를 활용하여 블로그에 기사를 쓰세요. "
                "출판물에는 온라인 리서처가 언급한 출처에 대한 링크가 포함되어야 합니다. "
                "최종 답변은 최소 3단락의 전체 블로그 게시물이어야 합니다. ",
    expected_output="블로그 아티클",    
    agent=blog_manager
)

In [34]:
task3 = Task(
    description="조사 결과를 사용하여 트윗을 작성하세요. "
                "최종 답변은 반드시 전체 트윗이어야 합니다. ",
    expected_output="Tweet",
    agent=social_media_manager
)

In [35]:
task4 = Task(
    description="블로그 관리자와 소셜 미디어 관리자의 최종 결과물을  "
                "꼼꼼히 검토하고 조화시켜 최종 출판물의 "
                "응집력과 우수성을 보장합니다. "
                "완료되면 최종 보고서를 게시합니다. ",
    expected_output="최종 보고서",
    agent=content_marketing_manager
)

## 에이전트 Crew 생성

In [36]:
crew = Crew(
    agents=[online_researcher, blog_manager, social_media_manager, content_marketing_manager],
    tasks=[task1, task2, task3, task4]
)

## 앱 실행 전 주의사항
* GPT-4로 이와 같은 멀티 에이전트 앱을 실행하는 것은 기존 앱을 실행하는 것보다 비용이 많이 발생합니다.
* LangSmith에서 총 비용에 보세요. 

In [37]:
result = crew.kickoff()

# Agent: 온라인 리서처
## Task: Agentic Behavior에 대한 보고서를 작성해 주세요. Agentic Behavior에 대한 조사가 끝나면  블로그 관리자에게 결과를 전달하여 최종 블로그 기사를 생성해 주세요. 완료되면 소셜 미디어 관리자에게 전달하여 해당 주제에 대한 트윗을 작성해 주세요. 
# Agent: 블로그 관리자
## Task: What are the latest trends and key studies regarding Agentic Behavior that I should include in the report?


# Agent: 블로그 관리자
## Final Answer: 
In recent years, the study of Agentic Behavior has gained significant attention across various fields, including psychology, education, sociology, and organizational behavior. Here are some of the latest trends and key studies that should be included in your report:

1. **Definition and Relevance**: Agentic behavior is characterized by an individual's ability to act independently and make choices that reflect their values and interests. This concept aligns closely with autonomy, self-regulation, and empowerment, which are crucial in various settings such as educational environments, workplaces, and personal development.

2. **Recent Trends**:
  

In [17]:
from IPython.display import Markdown
Markdown(result.raw)

# Final Report on Agentic Behavior: Definition, Characteristics, and Significance

## Understanding Agentic Behavior
Agentic behavior refers to the capacity of individuals to act autonomously, make intentional choices, and influence their environments. This concept is pivotal in psychology, as it highlights the active role people play in shaping their own lives and relationships. 

## Key Characteristics of Agentic Behavior
- **Self-Awareness**: Recognizing one's strengths and weaknesses for personal growth. 
- **Intentionality**: Setting and pursuing goals deliberately, often illustrated by the SMART criteria (Specific, Measurable, Achievable, Relevant, Time-bound).
- **Responsibility**: Holding oneself accountable for actions and decisions.

These traits enhance personal development and promote healthier relationships through open communication and mutual respect.

## Examples and Significance
- A student seeking resources exemplifies agentic behavior, showcasing proactivity in learning.
- In professional contexts, team members initiating discussions on improvements reflect this behavior.

The significance of agentic behavior is profound; it contributes to psychological well-being and resilience, fostering a sense of control over one’s life and enriching interpersonal relationships by encouraging balanced dynamics.

## Conclusion
Understanding and fostering agentic behavior can lead to substantial benefits in personal growth and relationships. By promoting agency, individuals can navigate their environments more effectively, leading to more fulfilling interactions. 

For further reading on agentic behavior and its implications, consider exploring sources from [Verywell Mind](https://www.verywellmind.com), [Psychology Today](https://www.psychologytoday.com), [Wiley Online Library](https://onlinelibrary.wiley.com), [ResearchGate](https://www.researchgate.net), and the [Journal of Social Psychology](https://www.tandfonline.com/toc/vjsp20/current).

🌟 Unlock your potential with agentic behavior! Discover how self-awareness, intentionality, and responsibility can transform your life and relationships. Dive into our latest article to learn more about this empowering concept in psychology! 💪✨ #AgenticBehavior #Psychology #SelfImprovement #PersonalGrowth #HealthyRelationships

## LangSmith를 살펴보세요
* 프로젝트가 많은 토큰을 사용하는지 확인하세요.
* 각 에이전트가 모니터링되는지 확인하세요.
* 각 에이전트를 클릭하여 입력 및 출력을 확인하세요.

https://smith.langchain.com/